In [1]:
import sys, os 
import numpy as np
import math
import glob

from copy import deepcopy

import matplotlib.pyplot as plt
from itertools import product
import importlib

if importlib.util.find_spec('matplotlib'):
    import matplotlib
    import matplotlib.pyplot as plt
    from matplotlib.font_manager import FontProperties
    # matplotlib.font_manager._rebuild()

if importlib.util.find_spec('ipywidgets'):
    from ipywidgets.widgets import IntSlider
    from ipywidgets import interact

In [16]:
# Useful functions

# compute Value at Risk
def get_var(arr, alpha, atoms):
    var = 0
    cum_p = 0
    for j, atom in enumerate(arr):
        cum_p += atom
        if cum_p>= alpha:
            var = atoms[j]
            break
    return var

# compute conditional value at Risk
def get_cvar(arr, alpha, atoms, var):
    cvar = var
    expected_c = 0 
    for i, j in enumerate(arr):
        if arr[i] > 0:
            expected_c += j * max(0,atoms[i]-var)
    cvar+= 1/(1- alpha) * expected_c
    return cvar

def compute_cvar(arr, s, alpha, atoms, b_atoms):
    temp_cvar= np.zeros(b_atoms.shape)
    
    for idx_b in range(len(b_atoms)):
        expected_c = 0 
        for i, j in enumerate(arr[idx_b][int(policy[s][idx_b])]):
            if j > 0:
                expected_c += j * max(0,atoms[i]-b_atoms[idx_b])
                
        temp_cvar[idx_b]=b_atoms[idx_b]+ 1/(1- alpha) * expected_c
    print(temp_cvar)
    idx_min_b = np.argmin(temp_cvar)
    return idx_min_b, temp_cvar[idx_min_b]
        
    

In [3]:
def get_closest_b(b):
    # make sure it is in range
    new_b = max(b_atoms[0], min(b_atoms[-1], b))
    index = new_b/delta_b
    l = math.floor(index); u= math.ceil(index)
    
    # TODO What do we choose NOW?
    # more harsh policy -> choose l
    return l

# E[[dist-b]+]
def get_magic_number(dist, b_idx):
    res = 0
    for j in range(natoms):
        res += dist[j] * max(0, (atoms[j] - b_atoms[b_idx]))
    
    return res
    
def compute_dist(s, action, b, r):
    m= np.zeros([natoms])
    sum_p= np.zeros([natoms])
                       
    # get closest b based on transition[2]
    temp_b = (b-r)/ beta
    print(f" new_b : {temp_b}, b:{b}, r:{r}, beta:{beta}")
    b_idx = get_closest_b(temp_b) 
    
    for transition in trans_p[s][action]:
        for a in range(natoms):
            # probabilities
            sum_p[a] += transition[0] * dist[(transition[1]-1)][b_idx][action][a]

    #print(sum_p)

    for a in range(natoms):
        for transition in trans_p[s][action]:
            # support adjustment
            cost = max(atoms[0], min(atoms[-1], r+ beta*atoms[a]))

            index = cost/delta_a
            l = math.floor(index); u= math.ceil(index)

            #print(round(index, 3),"atoms[a]:", atoms[a], "sum_p[a]:", sum_p[a], "cost:",transition[2]+ beta*atoms[a], "discount: ", beta)

            if (l != u):
                m[l] += sum_p[a] * (u -index) * transition[0]
                m[u] += sum_p[a] * (index-l) * transition[0]
            else:
                m[l] += sum_p[a] * transition[0]
    
    return m, get_magic_number(m, b_idx), b_atoms[b_idx], b_idx

In [4]:
def value_iteration(sigma, trans_p, nstates=4, T=4, beta= 1):
    
    temp_dist = np.zeros([nstates, nb, nactions, natoms])
    policy = np.zeros([nstates, nb])

    for i in range(T):
        #update initialize temp distribution for t+1
        for s in range(nstates):
            for idx_b in range(nb):
                for idx_a, arr in enumerate(dist[s][idx_b]):
                    temp_dist[s][idx_b][idx_a] = deepcopy(arr)
                    
        for s in range(nstates):
            for idx_b in range(nb): 
                print(f"\n------------- t:{i}, state: {s} ---------------")  
                temp_magic = np.ones([nactions]) * np.inf
                for idx_a in range(len(actions[s])):
                    print(f"---- action: {idx_a}------")
                    m,magic, new_b, new_idx_b = compute_dist(s, idx_a, b_atoms[idx_b], reward[s][idx_a])
                    temp_magic[idx_a] = magic
                    print(f"s: {s}, newb:{new_b}, new_idx_b:{new_idx_b}, idx_a{idx_a}, m: {m}")
                    temp_dist[s][idx_b][idx_a] = deepcopy (m) # should this be new_b or b?


                min_a = np.argmin(temp_magic)
                policy[s][idx_b] =min_a
                
            print(f"-------temp_magic: {temp_magic}, min_a:{min_a}")
        
        #update distribution
        for s in range(nstates):
            for idx_b in range(nb):
                for idx_a, arr in enumerate(temp_dist[s][idx_b]):
                    dist[s][idx_b][idx_a] = deepcopy(arr)

        #print(f"\n------------- dist for state:{0} i: {i}, b:{temp_b[0]} ---------------\n {dist[0][get_closest_b(temp_b[0])]}")  
    
    return dist, policy

### Model 1

In [33]:
nstates = 4
actions = [(1,2), (1,),(1,),(1,)]
trans_p = [[[(0.5,2, 0), (0.5,4,0)], [(1.0, 3, 0.5)]], [[(1,2,0)]], [[(1,3,0)]], [[(1,4,2)]]]
reward = [[0,0.5], [0], [0], [2]] # for each state-action pair

In [34]:
natoms = 16
beta = 1
delta_a = 0.2
atoms = np.array([0+round(i*delta_a, 2) for i in range(natoms)])

In [35]:
c_n = 0
visited=[]
# sigma = [[1,1,1,1], [1,1,1,1], [1,1,1,1], [1,1,1,1]] # sigma 1
sigma = [[2,1,1,1], [2,1,1,1], [2,1,1,1], [2,1,1,1]] # sigma 2  
desired_cost= 0

In [36]:
nb = 16
delta_b = 0.2
b_atoms = np.array([0+round(i*delta_b, 2) for i in range(nb)])
dist = np.zeros([nstates, nb, nactions, natoms])
for s in range(nstates):
    for b in range (nb):
        for idx_a in range(nactions):
            dist[s][b][idx_a][0]= 1.0
    
#dist

In [37]:
vals, policy= value_iteration(sigma, trans_p, beta=beta, nstates=nstates, T=2)


------------- t:0, state: 0 ---------------
---- action: 0------
 new_b : 0.0, b:0.0, r:0, beta:1
s: 0, newb:0.0, new_idx_b:0, idx_a0, m: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
---- action: 1------
 new_b : -0.5, b:0.0, r:0.5, beta:1
s: 0, newb:0.0, new_idx_b:0, idx_a1, m: [0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]

------------- t:0, state: 0 ---------------
---- action: 0------
 new_b : 0.2, b:0.2, r:0, beta:1
s: 0, newb:0.2, new_idx_b:1, idx_a0, m: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
---- action: 1------
 new_b : -0.3, b:0.2, r:0.5, beta:1
s: 0, newb:0.0, new_idx_b:0, idx_a1, m: [0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]

------------- t:0, state: 0 ---------------
---- action: 0------
 new_b : 0.4, b:0.4, r:0, beta:1
s: 0, newb:0.4, new_idx_b:2, idx_a0, m: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
---- action: 1------
 new_b : -0.09999999999999998, b:0.4, r:0.5, beta:1
s: 0, newb:0.0, new_idx_b:0, idx_a

In [176]:
alpha=0.5
print(f" VaR sigma {sigma}, alpha:{alpha}")
var=get_var(dist[0], alpha, atoms)
print(var)

 VaR sigma [[2, 1, 1, 1], [2, 1, 1, 1], [2, 1, 1, 1], [2, 1, 1, 1]], alpha:0.5
0.4


In [38]:
alpha=0.5
print(f" CvaR alpha:{alpha}")
print(compute_cvar(vals[0], 0, alpha, atoms, b_atoms))      

 CvaR alpha:0.5
[1.  0.8 0.6 0.6 0.8 1.  1.2 1.4 1.6 1.8 2.  2.2 2.4 2.6 2.8 3. ]
(2, 0.6)


In [117]:
cur_s = [0]
cur_p = [1.0]
c =np.zeros([T,natoms])
for i in range(T):
    for j,s in enumerate(cur_s):
        action = sigma[i][s]-1
        news = set()
        newp = []
        print(trans_p[s][action])
        for transition in trans_p[s][action]:
            cost = transition[2]*pow(beta,i)
            index = round(cost/delta_a, 2)
            print(f"cost : {cost}, index :{cost/delta_a}, cur_p {cur_p[j]}")
            c[i][int(index)] += (transition[0] * cur_p[j])
            news.add(transition[1]-1)
            newp.append(transition[0])
    cur_s = list(news)
    cur_p = newp
    print(newp)
            
print(c)
print(atoms)
print(news)

[(0.5, 2, 0), (0.5, 4, 2)]
cost : 0.0, index :0.0, cur_p 1.0
cost : 2.0, index :10.0, cur_p 1.0
[0.5, 0.5]
[[0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.  0.  0.  0.  0. ]]
[0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8, 3.0]
{1, 3}


In [165]:
atoms

array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. , 2.2, 2.4,
       2.6, 2.8, 3. ])

In [39]:
policy

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.])

### Model 2

In [5]:
nstates = 3
actions = [(1,2), (1,),(1,)]
nactions = 2
trans_p = [[[(0.5,2, 0), (0.5,1,0)], [(1.0, 3, 0.5)]], [[(1,2,1)]], [[(1,3,0)]]]
reward = [[0,0.5], [1], [0]] # for each state-action pair

In [10]:
b_atoms

array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. , 2.2, 2.4,
       2.6, 2.8, 3. ])

In [7]:
natoms = 16
beta = 0.4
delta_a = 0.2
atoms = np.array([0+round(i*delta_a, 2) for i in range(natoms)])

In [8]:
# sigma = [[1,1,1,1], [1,1,1,1]] # sigma 1
sigma = [[2,1,1], [1,1,1]] # sigma 2  
desired_cost= 0

#### Initialize distribution array

In [9]:
nb = 16
delta_b = 0.2
b_atoms = np.array([0+round(i*delta_b, 2) for i in range(nb)])
dist = np.zeros([nstates, nb, nactions, natoms])
for s in range(nstates):
    for b in range (nb):
        for idx_a in range(nactions):
            dist[s][b][idx_a][0]= 1.0
    
#dist

In [12]:
vals, policy= value_iteration(sigma, trans_p, beta=0.4, nstates=nstates, T=2)


------------- t:0, state: 0 ---------------
---- action: 0------
 new_b : 0.0, b:0.0, r:0, beta:0.4
s: 0, newb:0.0, new_idx_b:0, idx_a0, m: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
---- action: 1------
 new_b : -1.25, b:0.0, r:0.5, beta:0.4
s: 0, newb:0.0, new_idx_b:0, idx_a1, m: [0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]

------------- t:0, state: 0 ---------------
---- action: 0------
 new_b : 0.5, b:0.2, r:0, beta:0.4
s: 0, newb:0.4, new_idx_b:2, idx_a0, m: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
---- action: 1------
 new_b : -0.7499999999999999, b:0.2, r:0.5, beta:0.4
s: 0, newb:0.0, new_idx_b:0, idx_a1, m: [0.  0.  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]

------------- t:0, state: 0 ---------------
---- action: 0------
 new_b : 1.0, b:0.4, r:0, beta:0.4
s: 0, newb:1.0, new_idx_b:5, idx_a0, m: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
---- action: 1------
 new_b : -0.24999999999999994, b:0.4, r:0.5, beta:0.4
s: 0, 

In [13]:
vals[0]

array([[[0.5, 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. , 0. ],
        [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. , 0. ]],

       [[0.5, 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. , 0. ],
        [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. , 0. ]],

       [[0.5, 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. , 0. ],
        [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. , 0. ]],

       [[0.5, 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. , 0. ],
        [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. , 0. ]],

       [[0.5, 0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. , 0. ],
        [0. , 0. , 0.5, 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
         0. , 0. , 0. , 0. ]],



In [30]:
atoms

array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. , 2.2, 2.4,
       2.6, 2.8, 3. ])

In [106]:
b_atoms

array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. , 2.2, 2.4,
       2.6, 2.8, 3. ])

In [32]:
alpha=0.5
print(f" VaR sigma {sigma}, alpha:{alpha}")
var=get_var(dist[0], alpha, atoms)
print(var)

 VaR sigma [[2, 1, 1], [1, 1, 1]], alpha:0.5
0.4


In [17]:
alpha=0.5
print(f" CvaR, alpha:{alpha}")
print(compute_cvar(vals[0], 0, alpha, atoms, b_atoms))    

 CvaR sigma [[2, 1, 1], [1, 1, 1]], alpha:0.5
[0.4 0.4 0.4 0.6 0.8 1.  1.2 1.4 1.6 1.8 2.  2.2 2.4 2.6 2.8 3. ]
(0, 0.4)
